# Word2Vec CBOW and Skipgram
## written by: [Jehwan Kim](github.com/kreimben)
## date: 10th Feb 2024 ~ 12th Feb 2024
## corpus data: [kaggle](https://www.kaggle.com/datasets/junbumlee/kcbert-pretraining-corpus-korean-news-comments?resource=download)

In [ ]:
import pickle

from icecream import ic

from src.util import *

# Data Preprocessing

According to datasets from kaggle, It is already cleaned data. So I will use it directly.

In [ ]:
# Dataset example
count = 10
data = get_data()
while count:
    ic(next(data))
    count -= 1

Of course there are great library `KoNLPy`, But I found more accurate library [`soynlp`](https://github.com/lovit/soynlp) so I will use that.
soynlp learns new words automatically and calculate words statistically.
soynlp use `Cohesion score`, `Branching Entropy` and `Accessor Variety` internally.

In [ ]:
# corpus = DoublespaceLineCorpus("data.txt", iter_sent=True)
# len(corpus)

In [ ]:
# # to save the object cuz it consumes a lot of times!
# with open('data.pkl', 'wb') as f:
#     pickle.dump(corpus, f)

In [ ]:
# load data object from pickle
# if you don't have a txt file data from kaggle, uncomment it and load the object.
with open('data.pkl', 'rb') as f:
    corpus = pickle.load(f)

In [ ]:
i = 0
for d in corpus:
    print(i, d)
    i += 1
    if i > 4:
        break

In [ ]:
# word_extractor = WordExtractor()
# word_extractor.train(corpus)

In [ ]:
# # to save the trained object
# with open('word_extractor.pkl', 'wb') as f:
#     pickle.dump(word_extractor, f)

In [ ]:
# to load the trained object
with open('word_extractor.pkl', 'rb') as f:
    word_extractor = pickle.load(f)

In [ ]:
# to check the score of each method.
word_score = word_extractor.extract()

## Cohension
Cohesion은 문자열을 글자단위로 분리하여 부분문자열(substring)을 만들 때 왼쪽부터 문맥을 증가시키면서 각 문맥이 주어졌을 때 그 다음 글자가 나올 확률을 계산하여 누적곱을 한 값이다.

![](img1.png)

예를 들어 “연합뉴스가”라는 문자열이 있는 경우, 각 부분문자열의 cohesion은 다음과 같다. 한 글자는 cohesion을 계산하지 않는다.

![](img2.png)

하나의 단어를 중간에서 나눈 경우, 다음 글자를 예측하기 쉬우므로 조건부확률의 값은 크다. 하지만 단어가 종료된 다음에 여러가지 조사나 결합어가 오는 경우에는 다양한 경우가 가능하므로 조건부확률의 값이 작아진다. 따라서 cohesion값이 가장 큰 위치가 하나의 단어를 이루고 있을 가능성이 높다.

In [ ]:
word_score["핵"].cohesion_forward

In [ ]:
word_score["핵실"].cohesion_forward

In [ ]:
word_score["핵실험"].cohesion_forward

In [ ]:
word_score["핵실험은"].cohesion_forward

In [ ]:
word_score["핵실험이"].cohesion_forward

## Branching Entropy
Branching Entropy는 조건부 확률의 값이 아니라 확률분포의 엔트로피값을 사용한다. 
만약 하나의 단어를 중간에서 끊으면 다음에 나올 글자는 쉽게 예측이 가능하다. 
즉, 여러가지 글자 중 특정한 하나의 글자가 확률이 높다. 따라서 엔트로피값이 0에 가까운 값으로 작아진다. 
하지만 하나의 단어가 완결되는 위치에는 다양한 조사나 결합어가 올 수 있으므로 여러가지 글자의 확률이 비슷하게 나오고 따라서 엔트로피값이 높아진다.

In [ ]:
word_score["핵"].right_branching_entropy

In [ ]:
# '핵실' 다음에는 항상 '험'만 나온다.
word_score["핵실"].right_branching_entropy

In [ ]:
word_score["핵실험"].right_branching_entropy

In [ ]:
word_score["핵실험은"].right_branching_entropy

# Accessor Variety
Accessor Variety는 확률분포를 구하지 않고 단순히 특정 문자열 다음에 나올 수 있는 글자의 종류만 계산한다. 
글자의 종류가 많다면 엔트로피가 높아지리 것이라고 추정하는 것이다.

In [ ]:
word_score["핵"].right_accessor_variety

In [ ]:
# '핵실' 다음에는 항상 '험'만 나온다.
word_score["핵실"].right_accessor_variety

In [ ]:
word_score["핵실험"].right_accessor_variety

In [ ]:
word_score["핵실험은"].right_accessor_variety

We can assume the corpus is from social network and better `Max Score` tokenising for this corpus cuz it's not cultured writing.

# Max Score Tokenising

최대 점수 토큰화(max score tokenizing)는 띄어쓰기가 되어 있지 않는 긴 문자열에서 가능한 모든 종류의 부분문자열을 만들어서 가장 점수가 높은 것을 하나의 토큰으로 정한다. 
이 토큰을 제외하면 이 위치를 기준으로 전체 문자열이 다시 더 작은 문자열들로 나누어지는데 이 문자열들에 대해 다시 한번 가장 점수가 높은 부분문자열을 찾는 것을 반복한다. 

In [ ]:
from soynlp.tokenizer import MaxScoreTokenizer

scores = {word: score.cohesion_forward for word, score in word_score.items()}

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("안전성에문제있는스마트폰을휴대하고탑승할경우에압수한다")